In [1]:
import numpy as np
import cv2
import time
import os
import concurrent.futures as multithreading

In [2]:
class Video:
    
    @staticmethod
    def compress(filename, output_dst=None, fraction=0.01):

        def get_indices(height, width, fraction):

            def to_2d(index):
                return (index // width, index % width)

            samples = np.random.permutation(width * height)[:int(width * height * fraction)]

            indices = [to_2d(index) for index in samples]

            return tuple(zip(*indices))


        def sample_frame(frame, fraction):
            height, width, nb_channels = frame.shape

            indices = get_indices(height, width, fraction)

            return np.expand_dims(frame[indices], axis=0), np.expand_dims(indices, axis=0)


        cap = cv2.VideoCapture(filename)
        framerate = np.array(cap.get(cv2.cv2.CAP_PROP_FPS))

        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        start_time = time.time()

        ret, frame = cap.read()
        dimensions = np.array(frame.shape)

        video, indices = sample_frame(frame, fraction)
        curr_frame = 1

        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            sampled_frame, sampled_indices = sample_frame(frame, fraction)

            video = np.vstack((video, sampled_frame))
            indices = np.vstack((indices, sampled_indices))

            curr_frame += 1
            print('Compressing Video: ' + str(int(100 * curr_frame / num_frames)) + 
                  '%\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds', end='\r')

        cap.release()

        print('Compressing Video: 100%\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds')

        if output_dst is None:
            output_dst = '../Results/CompressedVideos/' + filename.split('/')[-1].split('.')[0]
        
        np.savez_compressed(
            output_dst, 
            video=video, 
            indices=indices, 
            dimensions=dimensions, 
            framerate=framerate
        )
    
    @staticmethod
    def psnr(original, reconstructed, color_channels='bgr'):
        cap_original = cv2.VideoCapture(original)
        cap_reconstructed = cv2.VideoCapture(reconstructed)
        
        num_frames_original = int(cap_original.get(cv2.CAP_PROP_FRAME_COUNT))
        num_frames_reconstructed = int(cap_reconstructed.get(cv2.CAP_PROP_FRAME_COUNT))
        num_frames = min(num_frames_original, num_frames_reconstructed)
        
        psnr_list = []
        max_pixel_value = np.iinfo(np.uint8).max
        
        start_time = time.time()
        curr_frame = 0
        
        while cap_original.isOpened() and cap_reconstructed.isOpened():
            ret_original, frame_original = cap_original.read()
            ret_reconstructed, frame_reconstructed = cap_reconstructed.read()
            
            if (not ret_original) or (not ret_reconstructed):
                break
            
            height = frame_original.shape[0]
            width = frame_original.shape[1]
            frame_mse = 0
            
            if color_channels == 'bgr':
                nb_channels = frame_original.shape[2]
                total_squared_error = np.sum(np.square(frame_original - frame_reconstructed))
                frame_mse = total_squared_error / (width * height * nb_channels)
            
            elif color_channels == 'ycbcr':
                frame_original = cv2.cvtColor(frame_original, cv2.COLOR_BGR2YCrCb)
                frame_reconstructed = cv2.cvtColor(frame_reconstructed, cv2.COLOR_BGR2YCrCb)
                total_squared_error = np.sum(np.square(frame_original[:,:,0] - frame_reconstructed[:,:,0]))
                frame_mse = total_squared_error / (width * height)
            
            if frame_mse != 0: # To avoid counting black frames
                frame_psnr = 10 * np.log10((max_pixel_value ** 2) / frame_mse)
                psnr_list.append(frame_psnr)
            
            curr_frame += 1
            print('Computing PSNR: ' + str(int(100 * curr_frame / num_frames)) +
                  '%\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds', end='\r')
        
        psnr = np.mean(psnr_list)
        
        print('Computing PSNR: 100%\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds')
        
        cap_original.release()
        cap_reconstructed.release()
        
        return psnr


In [3]:
class Kernels:
    
    @staticmethod
    def kernel_2d(kernel_size, sigma):
        center = kernel_size // 2
        kernel = np.fromfunction(
            lambda x, y: 
                np.exp( -0.5 * ((x - center) ** 2 + (y - center) ** 2) / (sigma ** 2)), 
            (kernel_size, kernel_size), 
            dtype=float)

        return kernel
    
    @staticmethod
    def kernel_3d(kernel_size, sigma, num_kernels, sigma_time):
        center = kernel_size // 2
        kernels = np.fromfunction(
            lambda t, x, y: 
                np.exp( -0.5 * (
                                 (((x - center) ** 2 + (y - center) ** 2) / (sigma ** 2)) + 
                                  ((t ** 2) / (sigma_time ** 2))
                               )
                      ), 
            (num_kernels, kernel_size, kernel_size), 
            dtype=float
        )
        return kernels


In [4]:
class CompressedVideo:
    
    def __init__(self, filename):
        
        file_dict = np.load(filename)
        
        self.filename = filename.split('_compressed.npz')[0]
        self.height = file_dict['dimensions'][0]
        self.width = file_dict['dimensions'][1]
        self.nb_channels = file_dict['dimensions'][2]
        self.video = file_dict['video']
        self.indices = file_dict['indices']
        self.num_frames = self.video.shape[0]
        self.framerate = file_dict['framerate']
        
        self.defaults = {}
        self.defaults['sigma']           = np.sqrt((self.height * self.width) / 
                                                   (self.video[0].shape[0] * np.pi))
        self.defaults['kernel_size']     = 2 * (int(0.5 + 3 * self.defaults['sigma']) + 2) + 1
        self.defaults['num_time_frames'] = 15
        self.defaults['output_dst']      = self.filename + '_reconstructed.mov'
        self.defaults['multithreaded']   = True
    
    
    def reconstruct(self, algorithm='efan2d', **kwargs):
        
        if 'kernel_size' in kwargs:
            kernel_size = kwargs['kernel_size']
        else:
            kernel_size = self.defaults['kernel_size']
        
        if 'sigma' in kwargs:
            sigma = kwargs['sigma']
        else:
            sigma = self.defaults['sigma']
        
        if 'num_time_frames' in kwargs:
            num_time_frames = kwargs['num_time_frames']
        else:
            num_time_frames = self.defaults['num_time_frames']
        
        if 'output_dst' in kwargs:
            output_dst = kwargs['output_dst']
        else:
            output_dst = self.defaults['output_dst']
        
        if 'multithreaded' in kwargs:
            multithreaded = kwargs['multithreaded']
        else:
            multithreaded = self.defaults['multithreaded']
        
        
        def display_progress(frame_index, start_time, filter_times, complete=False):
            print('Reconstructing Video: ' + str(int(100 * frame_index / self.num_frames)) + '% ' + 
                  '\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds ' +
                  '\tFilter Time: '  + str(int(np.floor(np.sum(filter_times)))) + ' seconds', end='\r')
            
            if complete:
                print('')
        
        
        def prepare_frame(frame_index):
            frame = self.video[frame_index]
            indices = self.indices[frame_index]

            black_frame = np.zeros((self.height, self.width, self.nb_channels))
            black_frame[tuple(indices)] = frame

            new_channel = np.zeros((self.height, self.width, 1)) + 1e-15 # To avoid division by zero
            new_channel[tuple(indices)] = 1.0

            augmented_frame = np.append(black_frame, new_channel, axis=2)
            
            return augmented_frame
        
        
        def normalize_frame(frame):
            for i in range(self.nb_channels):
                frame[:,:,i] /= frame[:,:,-1]

            reconstructed_frame = (frame[:,:,:-1] + 0.5).astype(np.uint8)
            
            return reconstructed_frame
        
        
        def efan2d(output):
            """
            EFAN2D Algorithm
            """
            
            filter_times = []

            def filter_frame(frame_index, kernel):
                augmented_frame = prepare_frame(frame_index)

                st = time.time()
                filtered = cv2.filter2D(augmented_frame, -1, kernel, 
                                        borderType=cv2.BORDER_CONSTANT)
                filter_times.append(time.time() - st)
                
                return filtered
            
            
            def reconstruct_frame(frame_index, kernel):
                filtered = filter_frame(frame_index, kernel)
                
                reconstructed_frame = normalize_frame(filtered)
                # Not sure about this, paper formula for reconstruction does not include it
                #reconstructed_frame[tuple(indices)] = frame
                
                return reconstructed_frame
            
            
            start_time = time.time()
            
            kernel = Kernels.kernel_2d(kernel_size, sigma)
            
            display_progress(0, start_time, filter_times)
            
            for frame_index in range(self.num_frames):
                output.write(reconstruct_frame(frame_index, kernel))
                display_progress(frame_index, start_time, filter_times)
            
            display_progress(self.num_frames, start_time, filter_times, complete=True)


        def efan3d(output):
            """
            EFAN3D Algorithm
            """
            
            num_kernels = (num_time_frames + 1) // 2
            sigma_time = (num_kernels - 1) / 6
            
            filter_times = []
            
            def filter_frame(frame_index, kernels):
                augmented_frame = prepare_frame(frame_index)

                st = time.time()
                
                if multithreaded:
                    filtered_frame = [None for i in range(num_kernels)]

                    def apply_kernel(kernel_index):
                        filtered = cv2.filter2D(augmented_frame, -1, kernels[kernel_index], 
                                                borderType=cv2.BORDER_CONSTANT)
                        filtered_frame[kernel_index] = filtered

                    with multithreading.ThreadPoolExecutor(max_workers=num_kernels) as executor:
                        executor.map(apply_kernel, range(num_kernels))
                
                else:
                    filtered_frame = []
                    for kernel in kernels:
                        filtered = cv2.filter2D(augmented_frame, -1, kernel, 
                                                borderType=cv2.BORDER_CONSTANT)
                        filtered_frame.append(filtered)
                
                filter_times.append(time.time() - st)
                
                return filtered_frame
            
            
            def reconstruct_frame(frame_index, filtered_frames):
                filtered_sum = np.zeros((self.height, self.width, self.nb_channels + 1))
            
                nb_past_frames = min(frame_index, num_kernels - 1)
                for idx in range(nb_past_frames):
                    filtered_sum += filtered_frames[idx][nb_past_frames - idx]

                nb_future_frames = min(self.num_frames - frame_index, num_kernels)
                for idx in range(nb_future_frames):
                    filtered_sum += filtered_frames[min(frame_index, num_kernels - 1) + idx][idx]
                
                reconstructed_frame = normalize_frame(filtered_sum)
                
                return reconstructed_frame
            
            
            start_time = time.time()
            
            kernels = Kernels.kernel_3d(kernel_size, sigma, num_kernels, sigma_time)
            
            filtered_frames = []
            
            display_progress(0, start_time, filter_times)
            
            for frame_index in range(self.num_frames):
                
                if frame_index == 0:
                    for idx in range(num_kernels):
                        filtered_frames.append(filter_frame(idx, kernels))
                        
                elif frame_index < num_kernels:
                    filtered_frames.append(filter_frame((num_kernels - 1) + frame_index, kernels))
                    
                elif frame_index <= self.num_frames - num_kernels:
                    filtered_frames.append(filter_frame((num_kernels - 1) + frame_index, kernels))
                    filtered_frames = filtered_frames[1:]
                    
                else:
                    filtered_frames = filtered_frames[1:]
                
                output.write(reconstruct_frame(frame_index, filtered_frames))
                
                display_progress(frame_index, start_time, filter_times)
            
            display_progress(self.num_frames, start_time, filter_times)
        
        
        def adefan(output):
            pass
            
        
        output = cv2.VideoWriter(
                    output_dst, 
                    cv2.VideoWriter_fourcc('M','J','P','G'), 
                    self.framerate, 
                    (self.width, self.height)
                 )
        
        if algorithm == 'efan2d':
            efan2d(output)
        elif algorithm == 'efan3d':
            efan3d(output)
        elif algorithm == 'adefan':
            adefan(output)
        
        output.release()
    

In [5]:
class Validation:
    
    resources_folder = '../Resources/Videos/'
    compressed_folder = '../Results/CompressedVideos/'
    reconstructed_folder = '../Results/ReconstructedVideos/'
    
    @staticmethod
    def compress_videos(motion='both'):
        
        def compress_folder_content(folder):
            suffix = folder + '/'
            in_folder = Validation.resources_folder + suffix
            out_folder = Validation.compressed_folder + suffix
            
            for video in os.listdir(in_folder):
                video_name = video.split('.')[0]
                Video.compress(in_folder + video, out_folder + video_name)
        
        if motion == 'absent' or motion == 'both':
            compress_folder_content('MotionAbsent')
        
        if motion == 'present' or motion == 'both':
            compress_folder_content('MotionPresent')
    
    @staticmethod
    def reconstruct_videos(algorithm='efan2d', motion='both', extension='.mp4'):
        
        def reconstruct_folder_content(folder):
            suffix = folder + '/'
            algorithm_folder = algorithm.upper() + '/'
            in_folder = Validation.compressed_folder + suffix
            out_folder = Validation.reconstructed_folder + algorithm_folder + suffix
            
            for video in os.listdir(in_folder):
                video_name = video.split('.')[0]
                out_filename = video_name + extension
                compressed_video = CompressedVideo(in_folder + video)
                compressed_video.reconstruct(algorithm=algorithm, output_dst=out_folder+out_filename)
        
        if motion == 'absent' or motion == 'both':
            reconstruct_folder_content('MotionAbsent')

        if motion == 'present' or motion == 'both':
            reconstruct_folder_content('MotionPresent')
    
    @staticmethod
    def compute_mean_psnr(algorithm='efan2d', motion='absent', color_channels='bgr'):
        
        def compute_folder_mean_psnr(folder):
            psnr_list = []
            
            suffix = folder + '/'
            original_folder = Validation.resources_folder + suffix
            algorithm_folder = algorithm.upper() + '/'
            reconstructed_folder = Validation.reconstructed_folder + algorithm_folder + suffix
            
            for original, reconstructed in zip(os.listdir(original_folder), os.listdir(reconstructed_folder)):
                psnr = Video.psnr(original_folder + original, 
                                  reconstructed_folder + reconstructed, 
                                  color_channels=color_channels)
                psnr_list.append(psnr)
            
            return np.mean(psnr_list)
        
        if motion == 'absent':
            return compute_folder_mean_psnr('MotionAbsent')
        
        if motion == 'present':
            return compute_folder_mean_psnr('MotionPresent')


In [5]:
Video.compress('Resources/VideoSamples/Beach.mp4', output_dst='../Results/Beach_compressed')

Compressing Video: 100%	Time Elapsed: 42 seconds


In [6]:
beach = CompressedVideo('../Results/Beach_compressed.npz')

In [7]:
beach.reconstruct(algorithm='efan2d', output_dst='../Results/beach_ef2.mov')

Reconstructing Video: 100% 	Time Elapsed: 149 seconds 	Filter Time: 84 seconds


In [8]:
beach.reconstruct(algorithm='efan3d', multithreaded=True, output_dst='../Results/beach_ef3.mov')

In [9]:
beach.reconstruct(algorithm='efan3d', multithreaded=False, output_dst='../Results/beach_ef3singlethreaded.mov')

In [10]:
Video.compress('Resources/VideoSamples/Seal.mp4')

Compressing Video: 100%	Time Elapsed: 1 seconds


In [13]:
seal = CompressedVideo('../Results/Seal_compressed.npz')

In [14]:
seal.reconstruct(algorithm='efan2d', output_dst='../Results/seal_ef2.mov')

Reconstructing Video: 100% 	Time Elapsed: 11 seconds 	Filter Time: 6 seconds


In [15]:
seal.reconstruct(algorithm='efan3d', multithreaded=True, output_dst='../Results/seal_ef3.mov')

In [16]:
seal.reconstruct(algorithm='efan3d', multithreaded=False, output_dst='../Results/seal_ef3singlethreaded.mov')

Using smaller resolutions seems much more reasonnable given the computational costs of hd videos.
Using multithreading for EFAN3D leads to a speedup of a factor 2 in runtime.

In [29]:
Validation.compress_videos()

Compressing Video: 100%	Time Elapsed: 6 seconds
Compressing Video: 100%	Time Elapsed: 0 seconds
Compressing Video: 100%	Time Elapsed: 1 seconds
Compressing Video: 100%	Time Elapsed: 6 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 0 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 0 seconds
Compressing Video: 100%	Time Elapsed: 1 seconds
Compressing Video: 100%	Time Elapsed: 4 seconds
Compressing Video: 100%	Time Elapsed: 3 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 3 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 3 seconds
Compressing Video: 100%	Time Elapsed: 2 seconds
Compressing Video: 100%	Time Elapsed: 3 seconds
Compressing Video: 100%	Time Elapsed: 1 

In [45]:
Validation.reconstruct_videos(algorithm='efan2d')

Reconstructing Video: 100% 	Time Elapsed: 23 seconds 	Filter Time: 13 seconds
Reconstructing Video: 100% 	Time Elapsed: 3 seconds 	Filter Time: 1 seconds
Reconstructing Video: 100% 	Time Elapsed: 7 seconds 	Filter Time: 4 seconds
Reconstructing Video: 100% 	Time Elapsed: 23 seconds 	Filter Time: 13 seconds
Reconstructing Video: 100% 	Time Elapsed: 9 seconds 	Filter Time: 5 seconds
Reconstructing Video: 100% 	Time Elapsed: 4 seconds 	Filter Time: 2 seconds
Reconstructing Video: 100% 	Time Elapsed: 10 seconds 	Filter Time: 5 seconds
Reconstructing Video: 100% 	Time Elapsed: 5 seconds 	Filter Time: 2 seconds
Reconstructing Video: 100% 	Time Elapsed: 9 seconds 	Filter Time: 5 seconds
Reconstructing Video: 100% 	Time Elapsed: 18 seconds 	Filter Time: 10 seconds
Reconstructing Video: 100% 	Time Elapsed: 14 seconds 	Filter Time: 7 seconds
Reconstructing Video: 100% 	Time Elapsed: 11 seconds 	Filter Time: 6 seconds
Reconstructing Video: 100% 	Time Elapsed: 17 seconds 	Filter Time: 9 seconds
Re

In [6]:
Validation.compute_mean_psnr(motion='absent')

Computing PSNR: 100%	Time Elapsed: 4 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 4 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 3 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 3 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds


32.73561705312805

In [7]:
Validation.compute_mean_psnr(motion='present')

Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 3 seconds
Computing PSNR: 100%	Time Elapsed: 3 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 4 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 3 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing 

32.99521054954046

In [8]:
Validation.compute_mean_psnr(motion='absent', color_channels='ycbcr')

Computing PSNR: 100%	Time Elapsed: 3 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds


32.73986462671497

In [9]:
Validation.compute_mean_psnr(motion='present', color_channels='ycbcr')

Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 0 seconds
Computing PSNR: 100%	Time Elapsed: 2 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing PSNR: 100%	Time Elapsed: 1 seconds
Computing 

33.13580115219883

In [29]:
##################################
##### Optimization Benchmark #####
##################################

height = 720
width = 1280
num_channels = 3
length = 38 #seconds
framerate = 24
num_frames = length * framerate
anchor_x = 400
anchor_y = 400
num_rand_pixels = (height * width) // 100

kernel_size = 71
kernel = Kernels.kernel_2d(kernel_size, 20)
r = (kernel_size - 1) // 2

###########################
##### Naive Operation #####
###########################

st = time.time()

kernel_dict = {}
for i in range(256):
    kernel_dict[i] = kernel * i

filtered = np.zeros((height, width, num_channels + 1));

for idx in range(num_rand_pixels // 100):
    k = kernel_dict[np.random.randint(0,256)]
    for c in range(num_channels + 1):
        for y in range(kernel_size):
            for x in range(kernel_size):
                filtered[anchor_x-r+y,anchor_y-r+x,c] += k[x,y]
tot_time_naive = (time.time() - st) * num_frames * 100
print('Naive python operations / array indexing:', np.around(tot_time_naive / 3600, 1), 'hours')

###########################
##### Numpy Optimized #####
###########################

st = time.time()

filtered = filtered = np.zeros((height, width, num_channels + 1));

kernel_dict = {}
for i in range(256):
    kernel_dict[i] = kernel * i

for i in range(num_rand_pixels):
    for c in range(num_channels + 1):
        filtered[anchor_y-r:anchor_y+r+1, anchor_x-r:anchor_x+r+1, c] += kernel_dict[np.random.randint(0,256)]

tot_time_numpy = (time.time() - st) * num_frames
print('Numpy (C) operations / array indexing:', np.around(tot_time_numpy / 60, 1), 'minutes')
print('Numpy speedup:', np.around(tot_time_naive / tot_time_numpy, 1), 'times faster')

Naive python operations / array indexing: 29.6 hours
Numpy (C) operations / array indexing: 9.6 minutes
Numpy speedup: 185.0 times faster


The equivalent of the MATLAB / C++ implementation would take over 29 hours to reconstruct the video using python operations and almost 10 minutes using numpy operations. Since numpy is already written in C there isn't much more to do to speed up the additions / multiplications / array indexing. Even using numpy this is still more than 4 times slower than using opencv to interpolate the value of each pixel (Probably because of the time gained by using filter2D which uses the FFT algorithm).

In [6]:
######################################
##### MATLAB / Python comparison #####
######################################

import scipy.io as sio

frame = cv2.imread('Resources/ImageComparison/lena.png')
width = frame.shape[1]
height = frame.shape[0]
nb_channels = frame.shape[2]

randvec = sio.loadmat('Resources/ImageComparison/randvec.mat')['randind'][0] - 1 # MATLAB indexing starts at 1

st = time.time() # Start counting time

indices = [(index % height, index // height) for index in randvec] # MATLAB is column major so we convert to row major
indices = tuple(zip(*indices))

black_frame = np.zeros((height, width, nb_channels))
black_frame[indices] = frame[indices]

new_channel = np.zeros((height, width, 1)) + 1e-15 # To avoid division by zero
new_channel[indices] = 1.0

augmented_frame = np.append(black_frame, new_channel, axis=2)

sigma = np.sqrt((height * width) / (len(indices[0]) * np.pi))
kernel_size = 2 * (int(0.5 + 3 * sigma) + 2) + 1

kernel = Kernels.kernel_2d(kernel_size, sigma)

filtered = cv2.filter2D(augmented_frame, -1, kernel, borderType=cv2.BORDER_CONSTANT)
for i in range(nb_channels):
    filtered[:,:,i] /= filtered[:,:,-1]
reconstructed_frame = (filtered[:,:,:-1] + 0.5).astype(np.uint8)

print('Total time elapsed', time.time() - st, 'seconds')

cv2.imwrite('Resources/ImageComparison/lena_python.png', reconstructed_frame);

Total time elapsed 0.052004098892211914 seconds


Our method takes the same amount of time as the MATLAB / C++ implementation to reconstruct the image

In [7]:
lena_matlab = cv2.imread('Resources/ImageComparison/lena_matlab.png')
lena_python = cv2.imread('Resources/ImageComparison/lena_python.png')

print('Number of different channel values:', np.count_nonzero(lena_matlab - lena_python))

Number of different channel values: 6


Only 6 channel values differ between the two images so the methods are equivalent (those values are probably because of different rounding / precision between python and C++)